## Description

This notebook is to investigate how different convergence properties related to each other. The goal is to find the best properties defined that are materials independent and can be used to predict the convergence of a PP. 

The properties that are investigated are:
- For pressure, compare the complex defined SSSP v1 residue volume and the vannila hydrostatic pressure
- For EOS metrics, compare nu wrt AE and nu with ref 200Ry. (Check and assure the guess that delta' and nu are correlated)
- Compare pressure and EOS metrics (nu ref 200Ry)
- Other pair see if those are correlated or not

What I think is, if I tuning the criteria of properties, there will be a cross from A > B to B > A. The different between if A, B are correlated or not is whether their will be a state where A, B are highly linearly correlated. 

The testing data is generated by running full convergence test in the grid of [20:5:200] Ry for all different properties calculation method, then can extract and construct the properties date from the output.
The tested PPs are Hg, Ga, N, Cs, Mn from gbrv, dojo, psl-paw-high and jth, in order to cover PPs from different generated code sources and different type of elements.

The AiiDA data is stored at group `SI/convergence-properties-compare`. 

In [1]:
from aiida import load_profile
import typing as t

load_profile("2023-08-07")

from aiida import orm

In [2]:
from aiida_sssp_workflow.workflows.convergence.pressure import helper_get_volume_from_pressure_birch_murnaghan
from aiida_sssp_workflow.calculations.calculate_metric import rel_errors_vec_length, _calcDelta
from aiida_sssp_workflow.calculations.calculate_bands_distance import get_bands_distance

paper_scan_list = [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 90, 100, 120, 150, 200]

def extract_data_scan_list1(node):
    real_scan_list = []
    for wf in node.called:
        if wf.process_label == 'ConvergenceCohesiveEnergyWorkChain':
            lst = []
            for wf2 in wf.called:

                if wf2.process_label == 'helper_cohesive_energy_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']
        
        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}
    
    for i, wf in enumerate(lst):
        cutoff = scan_list[i]

        i_para = wf.inputs.input_parameters.get_dict()
        r_para = wf.inputs.ref_parameters.get_dict()
    
        # Get the data
        # Ref_200_nu: the nu value w.r.t. the 200 Ry reference
        # Ref_200_deltap: the delta_p value w.r.t. the 200 Ry reference
        res = get_conv_data1(i_para, r_para)
        data[cutoff] = res

    return data, scan_list

def get_conv_data1(i_para, r_para) -> float:
    res_energy = i_para["cohesive_energy_per_atom"]
    ref_energy = r_para["cohesive_energy_per_atom"]
    absolute_diff = abs(res_energy - ref_energy) * 1000.0 # in meV
    relative_diff = abs((res_energy - ref_energy) / ref_energy) * 100

    return absolute_diff
    
def extract_data_scan_list2(node):
    real_scan_list = []
    for wf in node.called:
        if wf.process_label == 'ConvergenceCohesiveEnergyWorkChain':
            lst = []
            for wf2 in wf.called:
                if wf2.process_label == 'helper_cohesive_energy_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']
        
        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}

    for i, wf in enumerate(lst):
        cutoff = scan_list[i]

        i_para = wf.inputs.input_parameters.get_dict()
        r_para = wf.inputs.ref_parameters.get_dict()
    
        # Get the data
        # Ref_200_nu: the nu value w.r.t. the 200 Ry reference
        # Ref_200_deltap: the delta_p value w.r.t. the 200 Ry reference
        res = get_conv_data2(i_para, r_para)
        data[cutoff] = res

    return data, scan_list

def get_conv_data2(i_para, r_para) -> float:
    res_energy = i_para["cohesive_energy_per_atom"]
    ref_energy = r_para["cohesive_energy_per_atom"]
    absolute_diff = abs(res_energy - ref_energy) * 1000.0 # in meV
    relative_diff = abs((res_energy - ref_energy) / ref_energy) * 100

    return relative_diff



In [3]:
g = 'SI/convergence-properties-compare/DC'
gs_nodes = []
gs_nodes.extend(orm.Group.collection.get(label=g).nodes)
    
all_data1 = {}
all_data2 = {}
for node in gs_nodes:
    # give a node and the tuple of criteria
    # return the deducted cutoffs of A and B
    try:
        data1, scan_list1 = extract_data_scan_list1(node)
        data2, scan_list2 = extract_data_scan_list2(node)
        all_data1[node.pk] = {
            "data": data1,
            "scan_list": scan_list1    
        }
        all_data2[node.pk] = {
            "data": data2,
            "scan_list": scan_list2    
        }
    except Exception as e:
        #print(f"Error: {e}")
        #continue
        print(node.pk)
        raise e

Warning - the following cutoffs are missing from node 8292113: [25]
Warning - the following cutoffs are missing from node 8292113: [25]


In [4]:
def extract_cutoff(data, scan_list, criteria):
    """Extract the cutoff for pA and pB from a verification workchain

    Args:
        data (dict): the data extracted from the verification workchain
        scan_list (list): the list of cutoffs used in the verification workchain
        criteria (tuple): first element is the criteria for pA, second element is the criteria for pB

    Returns:
        tuple: the cutoff for pA and pB.
    """
    # Get the cutoff of pA and pB
    cut = 200
    for cutoff in reversed(scan_list):
        try: 
            p = data[cutoff]
        except:
            continue
        
        if p > criteria:
            break

        cut = cutoff

    return cut

In [5]:
def compute_cutoff(data12_tuple, criteria):
    cut_A_lst = []
    cut_B_lst = []

    all_data1 = data12_tuple[0]
    all_data2 = data12_tuple[1]
    criteria1 = criteria[0]
    criteria2 = criteria[1]

    for node_pk in all_data1:
        data = all_data1[node_pk]['data']
        scan_list = all_data1[node_pk]['scan_list']
        cut_A = extract_cutoff(data, scan_list, criteria1)
        cut_A_lst.append(cut_A)
    
    for node_pk in all_data2:
        data = all_data2[node_pk]['data']
        scan_list = all_data2[node_pk]['scan_list']
        cut_B = extract_cutoff(data, scan_list, criteria2)
        cut_B_lst.append(cut_B)
        
    return cut_A_lst, cut_B_lst

In [6]:
# Get data for plotting
cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(all_data1, all_data2), criteria=(0.1, 0.1))

In [7]:
import ipywidgets as ipw
import plotly.graph_objects as go

trace_corr_scatter = go.Scatter(x=cut_A_lst, y=cut_B_lst, mode='markers', name='cutoff correlation')
trace_xy_line = go.Scatter(x=[0, 200], y=[0, 200], name='x=y')
g = go.FigureWidget(data=[trace_corr_scatter, trace_xy_line])
g.layout.xaxis.title = 'cutoff pA'
g.layout.yaxis.title = 'cutoff pB'


In [10]:
pA_slider = ipw.FloatSlider(value=2, min=0.00, max=10.0, step=0.1, description='pA')
pB_slider = ipw.FloatSlider(value=0.1, min=0.00, max=2, step=0.1, description='pB')

def response(change):
    cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(all_data1, all_data2), criteria=(pA_slider.value, pB_slider.value))
    with g.batch_update():
        g.data[0].x = cut_A_lst
        g.data[0].y = cut_B_lst
        
pA_slider.observe(response, names="value")
pB_slider.observe(response, names="value")

slider_widgets = ipw.HBox([pA_slider, pB_slider])
app = ipw.VBox([slider_widgets, g])
app

In [12]:
# heatmap
import numpy as np

def compute_correlation(x, y):
    cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(all_data1, all_data2), criteria=(x, y))
    arr_A = np.array(cut_A_lst)
    arr_B = np.array(cut_B_lst)
    
    z = np.sum(np.abs(arr_A - arr_B) / 40.0)
    
    return z

xxs = np.linspace(0.0, 10, 201)
yys = np.linspace(0.0, 4, 201)
# get z map from xxs and yys
zzs = np.zeros((201, 201))
for i, x in enumerate(xxs):
    for j, y in enumerate(yys):
        zzs[j, i] = compute_correlation(x, y)
        
fig = go.FigureWidget(data=go.Heatmap(z=zzs, x=xxs, y=yys, zmax=10, zmin=0))
fig.layout.xaxis.title = 'cutoffA'
fig.layout.yaxis.title = 'cutoffB'
fig

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '3dd4d47c-5880-40be-bae3-a1e842aeec25',
              'x': array([ 0.  ,  0.05,  0.1 , ...,  9.9 ,  9.95, 10.  ]),
              'y': array([0.  , 0.02, 0.04, ..., 3.96, 3.98, 4.  ]),
              'z': array([[ 0.   , 32.   , 40.625, ..., 75.875, 75.875, 75.875],
                          [51.375, 25.875, 19.25 , ..., 24.5  , 24.5  , 24.5  ],
                          [58.875, 28.875, 22.25 , ..., 17.   , 17.   , 17.   ],
                          ...,
                          [83.5  , 51.5  , 42.875, ...,  9.125,  9.125,  9.125],
                          [83.625, 51.625, 43.   , ...,  9.25 ,  9.25 ,  9.25 ],
                          [83.625, 51.625, 43.   , ...,  9.25 ,  9.25 ,  9.25 ]]),
              'zmax': 10,
              'zmin': 0}],
    'layout': {'template': '...', 'xaxis': {'title': {'text': 'cutoffA'}}, 'yaxis': {'title': {'text': 'cutoffB'}}}
})